In [31]:
import json
from datetime import datetime, time

from pymongo.mongo_client import MongoClient
import bson

* A partir du fichier trips_.json, remplacez les espaces des clés par un underscore, puis mettez en base en changeant les chaîne de caractères qui concerne des grandeurs numériques par le type integer ou float et les champs de date ($date) doit être sous format datetime.datetime. Ainsi les champs préfixé '$number...' suivent  l'exemple ci-dessous(cherchez du côté de la librairie bson, il existe une fonction utilitaire, il faut aussi réduire la profondeur):

```json
{'tripduration': {'$numberInt': 379}}
```

en :

```json
{'tripduration': 379}
```



Après avoir mis en base et uniquement, assurez-vous qu'il n'y a pas de doublons, si il y en a, écrivez une fonction python permettant de les retirer (sans à avoir à faire de téléchargement)

De plus, des hypothèses d'incohérences ont été émises par les parties prenantes, il s'agit d'écrire pour chacune de ces hypothèses une fonction python qui va détecter les documents incriminés et les flagger avec un champs supplémentaire pour rectification/enquête ultérieure :

* Un vélo loué deux fois, mais la deuxième période de location démarre avant le rendu de la première période de location, ce qui n'est pas normal, ou tout autre chevanchement de période.
* Un utilisateur trop jeune (le service est réservé au + de 13 ans)
* La date de naissance n'est pas renseignée
* Des locations trop courtes (1 secondes)
* Des temps de location incohérents par rapport au start_time et au end_time


Faites un rapport détaillé des anomalies trouvées (nombres d'occurences)

Puis, après avoir fait un nettoyage et écarté les locations suspectes, répondez aux questions métier qui suivent :

* Quels sont les 5 trajets (start station → end station) les plus fréquents pour les utilisateurs de genre féminin ? (gender = 1)

* Quel est le nombre total de trajets par type d’utilisateur (Subscriber vs Customer) pour l’année 2023 ?

* Quelle est la durée moyenne des trajets par station de départ pour les trajets commençant entre 7h et 9h ?

* Quel est le top 3 des stations avec la plus forte augmentation du nombre de trajets entre deux années consécutives ?

* Pour chaque catégorie d’âge, quelle est la durée médiane des trajets ?

* Quels sont les utilisateurs les plus actifs (10%) et leur contribution au nombre total de trajets ?

* Quelle est la répartition des trajets par jour de la semaine (lundi, mardi, etc.) ?

* Quel est le temps moyen passé en trajet pour chaque genre, filtré sur les trajets de plus de 10 minutes ?

* Combien de trajets ont démarré dans une station donnée pendant les heures de pointe (ex. 7h-9h et 17h-19h) ?

* Quelles sont les 5 stations avec la plus grande diversité d’utilisateurs (nombre distinct d’user id) ?